In [1]:
import torch
from torch import nn
from torch.nn import init

In [2]:
# pytorch已进行默认初始化
net = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3, 1))


print(net)
X = torch.rand(2, 4)
Y = net(X).sum()

Sequential(
  (0): Linear(in_features=4, out_features=3, bias=True)
  (1): ReLU()
  (2): Linear(in_features=3, out_features=1, bias=True)
)


# 访问模型参数

In [3]:
print(type(net.named_parameters()))
for name, param in net.named_parameters():
    print(name, param.size())

<class 'generator'>
0.weight torch.Size([3, 4])
0.bias torch.Size([3])
2.weight torch.Size([1, 3])
2.bias torch.Size([1])


In [4]:
for name, param in net[0].named_parameters():
    print(name, param.size(), type(param))

weight torch.Size([3, 4]) <class 'torch.nn.parameter.Parameter'>
bias torch.Size([3]) <class 'torch.nn.parameter.Parameter'>


In [5]:
class MyModel(nn.Module):
    def __init__(self, **kwargs):
        super(MyModel, self).__init__(**kwargs)
        self.weight1 = nn.Parameter(torch.rand(20, 20))
        self.weight2 = torch.rand(20, 20)
        
    def forward(self, x):
        pass

n = MyModel()
for name, param in n.named_parameters():
    print(name)

weight1


In [6]:
weight_0 = list(net[0].parameters())[0]
print(weight_0.data)
print(weight_0.grad)  # 反向传播前梯度为None
Y.backward()  # 计算梯度
print(weight_0.grad)

tensor([[ 0.1070, -0.2323,  0.1678,  0.3559],
        [ 0.3961,  0.2930, -0.4152,  0.0037],
        [ 0.2175,  0.3505,  0.1657, -0.3449]])
None
tensor([[-0.0799, -0.0758, -0.0946, -0.1090],
        [-0.4256, -0.4036, -0.5035, -0.5807],
        [ 0.2998,  0.3262,  0.3278,  0.3643]])


# 初始化模型参数

In [7]:
for name, param in net.named_parameters():
    if 'weight' in name:
        init.normal_(param, mean=0, std=0.01)
        print(name, param.data)

0.weight tensor([[-0.0140,  0.0183,  0.0039, -0.0017],
        [ 0.0049,  0.0224, -0.0091, -0.0019],
        [ 0.0044,  0.0232,  0.0215,  0.0112]])
2.weight tensor([[-0.0050, -0.0071, -0.0089]])


In [8]:
for name, param in net.named_parameters():
    if 'bias' in name:
        init.constant_(param, val=0)
        print(name, param.data)

0.bias tensor([0., 0., 0.])
2.bias tensor([0.])


# 自定义初始化方法

In [9]:
def normal_(tensor, mean=0, std=1):
    # 不追踪其他的操作
    with torch.no_grad():
        return tensor.normal_(mean, std)

In [10]:
def init_weight_(tensor):
    with torch.no_grad():
        tensor.uniform_(-10, 10)
        tensor *= (tensor.abs() >= 5).float()
    
for name, param in net.named_parameters():
    if 'weight' in name:
        init_weight_(param)
        print(name, param.data)

0.weight tensor([[-0.0000,  0.0000, -9.4446,  0.0000],
        [ 0.0000,  5.7646,  7.7188,  8.1547],
        [ 0.0000, -0.0000, -0.0000, -0.0000]])
2.weight tensor([[0.0000, 0.0000, 9.2320]])


In [11]:
for name, param in net.named_parameters():
    if 'bias' in name:
        param.data += 1
        print(name, param.data)

0.bias tensor([1., 1., 1.])
2.bias tensor([1.])


# 共享模型参数

In [12]:
linear = nn.Linear(1, 1, bias=False)
net = nn.Sequential(linear, linear)
print(net)
for name, param in net.named_parameters():
    init.constant_(param, val=3)
    print(name, param.data)

Sequential(
  (0): Linear(in_features=1, out_features=1, bias=False)
  (1): Linear(in_features=1, out_features=1, bias=False)
)
0.weight tensor([[3.]])


In [13]:
print(id(net[0]) == id(net[1]))
print(id(net[0].weight) == id(net[1].weight))

True
True


In [14]:
x = torch.ones(1, 1)
y = net(x).sum()
print(y)
y.backward()  # 计算梯度
print(net[0].weight.grad)

tensor(9., grad_fn=<SumBackward0>)
tensor([[6.]])
